In [24]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
import numpy as np

# **One-Hot Encoding des logs**

In [25]:
import pandas as pd

print("Chargement du fichier CSV...")
df = pd.read_csv("all_logs.csv")
print(f"Fichier chargé avec {len(df)} lignes et {df.shape[1]} colonnes.")

print("Vérification des valeurs nulles avant suppression :")
print(df.isnull().sum())  # Affiche le nombre de valeurs nulles par colonne
print("Suppression des lignes avec des valeurs nulles...")
df_cleaned = df.dropna()  # Supprime toutes les lignes contenant des NaN

print(f"Nombre de lignes après suppression des valeurs nulles : {len(df_cleaned)}")

print("Vérification des valeurs nulles après suppression :")
print(df_cleaned.isnull().sum())  # Affiche le nombre de valeurs nulles par colonne

print("Vérification des types des colonnes après conversion :")
df_cleaned[["Date", "Hostname", "Process", "IdProcess", "Message"]] = df_cleaned[["Date", "Hostname", "Process", "IdProcess", "Message"]].astype(str)
print(df_cleaned[["Date", "Hostname", "Process", "IdProcess", "Message"]].dtypes)


Chargement du fichier CSV...
Fichier chargé avec 162046 lignes et 5 colonnes.
Vérification des valeurs nulles avant suppression :
Date         29350
Hostname     29350
Process      29350
IdProcess    94758
Message      29350
dtype: int64
Suppression des lignes avec des valeurs nulles...
Nombre de lignes après suppression des valeurs nulles : 67288
Vérification des valeurs nulles après suppression :
Date         0
Hostname     0
Process      0
IdProcess    0
Message      0
dtype: int64
Vérification des types des colonnes après conversion :
Date         object
Hostname     object
Process      object
IdProcess    object
Message      object
dtype: object


<ipython-input-25-07f9318208bc>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[["Date", "Hostname", "Process", "IdProcess", "Message"]] = df_cleaned[["Date", "Hostname", "Process", "IdProcess", "Message"]].astype(str)


In [26]:
# Appliquer le One-Hot Encoding à l'aide de pandas.get_dummies
print("Application du One-Hot Encoding...")
df_encoded = pd.get_dummies(df_cleaned, columns=["Date", "Hostname", "Process", "IdProcess", "Message"])

# Afficher le nombre de colonnes après l'encodage
print(f"Le DataFrame encodé contient désormais {df_encoded.shape[1]} colonnes.")


Application du One-Hot Encoding...
Le DataFrame encodé contient désormais 19578 colonnes.


# **Entrainement du modèle**

In [27]:
# Paramètres pour l'entraînement par batch
batch_size = 1000  # Taille du lot (par exemple, 1000 lignes par lot)
n_batches = len(df_encoded) // batch_size  # Nombre de lots
n_estimators = 50  # Nombre d'arbres pour Isolation Forest

In [28]:
# Initialiser le modèle Isolation Forest avec contamination="auto"
train_data, test_data = train_test_split(df_encoded, test_size=0.2, random_state=42)  # 80% train, 20% test
model = IsolationForest(n_estimators=n_estimators, contamination="auto", random_state=42)


# Initialiser le modèle Isolation Forest avec contamination="auto"
model = IsolationForest(n_estimators=n_estimators, contamination="auto", random_state=42)

# Mélanger les indices du DataFrame de façon aléatoire pour l'entraînement
train_indices = np.random.permutation(len(train_data))

In [29]:
# Entraînement du modèle sur chaque lot d'entraînement
print("Entraînement du modèle sur chaque batch du jeu d'entraînement...")
n_batches_train = len(train_data) // batch_size  # Nombre de lots d'entraînement
for i in range(n_batches_train):
    # Extraire un lot de données aléatoires
    start_idx = i * batch_size
    end_idx = start_idx + batch_size
    batch_indices = train_indices[start_idx:end_idx]
    batch = train_data.iloc[batch_indices]

    # Entraîner le modèle sur le lot
    model.fit(batch)

    # Afficher l'état d'avancement
    print(f"Batch {i + 1}/{n_batches_train} du jeu d'entraînement traité.")

# Si les données d'entraînement ne sont pas divisibles par batch_size, traiter le dernier lot restant
if len(train_data) % batch_size != 0:
    batch_indices = train_indices[n_batches_train * batch_size:]
    batch = train_data.iloc[batch_indices]
    model.fit(batch)
    print(f"Dernier batch du jeu d'entraînement traité.")

Entraînement du modèle sur chaque batch du jeu d'entraînement...
Batch 1/53 du jeu d'entraînement traité.
Batch 2/53 du jeu d'entraînement traité.
Batch 3/53 du jeu d'entraînement traité.
Batch 4/53 du jeu d'entraînement traité.
Batch 5/53 du jeu d'entraînement traité.
Batch 6/53 du jeu d'entraînement traité.
Batch 7/53 du jeu d'entraînement traité.
Batch 8/53 du jeu d'entraînement traité.
Batch 9/53 du jeu d'entraînement traité.
Batch 10/53 du jeu d'entraînement traité.
Batch 11/53 du jeu d'entraînement traité.
Batch 12/53 du jeu d'entraînement traité.
Batch 13/53 du jeu d'entraînement traité.
Batch 14/53 du jeu d'entraînement traité.
Batch 15/53 du jeu d'entraînement traité.
Batch 16/53 du jeu d'entraînement traité.
Batch 17/53 du jeu d'entraînement traité.
Batch 18/53 du jeu d'entraînement traité.
Batch 19/53 du jeu d'entraînement traité.
Batch 20/53 du jeu d'entraînement traité.
Batch 21/53 du jeu d'entraînement traité.
Batch 22/53 du jeu d'entraînement traité.
Batch 23/53 du jeu d

In [30]:
# Prédiction sur le jeu de test
print("Prédiction des anomalies sur le jeu de test...")
y_pred_test = np.zeros(len(test_data), dtype=int)

# Mélanger les indices du DataFrame de test
test_indices = np.random.permutation(len(test_data))

# Prédiction incrémentale sur les données de test
n_batches_test = len(test_data) // batch_size  # Nombre de lots de test
for i in range(n_batches_test):
    # Extraire un lot de données aléatoires
    start_idx = i * batch_size
    end_idx = start_idx + batch_size
    batch_indices = test_indices[start_idx:end_idx]
    batch = test_data.iloc[batch_indices]

    # Prédiction pour le lot actuel
    batch_pred = model.predict(batch)

    # Convertir les prédictions : -1 signifie anomalie, 1 signifie normal
    batch_pred = np.where(batch_pred == -1, 1, 0)

    # Ajouter les prédictions au tableau y_pred_test
    y_pred_test[start_idx:end_idx] = batch_pred

    print(f"Batch {i + 1}/{n_batches_test} du jeu de test prédit.")

# Si les données de test ne sont pas divisibles par batch_size, traiter le dernier lot restant
if len(test_data) % batch_size != 0:
    batch_indices = test_indices[n_batches_test * batch_size:]
    batch = test_data.iloc[batch_indices]
    batch_pred = model.predict(batch)
    batch_pred = np.where(batch_pred == -1, 1, 0)
    y_pred_test[n_batches_test * batch_size:] = batch_pred
    print(f"Dernière prédiction du batch du jeu de test effectuée.")

Prédiction des anomalies sur le jeu de test...
Batch 1/13 du jeu de test prédit.
Batch 2/13 du jeu de test prédit.
Batch 3/13 du jeu de test prédit.
Batch 4/13 du jeu de test prédit.
Batch 5/13 du jeu de test prédit.
Batch 6/13 du jeu de test prédit.
Batch 7/13 du jeu de test prédit.
Batch 8/13 du jeu de test prédit.
Batch 9/13 du jeu de test prédit.
Batch 10/13 du jeu de test prédit.
Batch 11/13 du jeu de test prédit.
Batch 12/13 du jeu de test prédit.
Batch 13/13 du jeu de test prédit.
Dernière prédiction du batch du jeu de test effectuée.


In [31]:
# Ajouter les prédictions au DataFrame de test
test_data["Anomaly"] = y_pred_test

# Afficher les anomalies détectées sur le jeu de test
anomalies_detectees = np.sum(y_pred_test)  # Compte le nombre d'anomalies (1 signifie anomalie)
print(f"Nombre d'anomalies détectées dans le jeu de test : {anomalies_detectees}")

Nombre d'anomalies détectées dans le jeu de test : 0
